# Setup Q&A and Personal Finance

## Setup Q&A

In [ ]:
# torch-scatter may take few minutes to install
!pip install pinecone-client sentence_transformers torch-scatter

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.4/179.4 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.0/108.0 kB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.4/300.4 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.6 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=2967735d36ebfe12f3da552f073bce7e39e9f905ad94d3a8c997c3c6d4f08777
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
  Created wheel for torch-scatter: filename=torch_scatter-2.1.2-cp310-cp310-linux_x86_64.whl size=495089 sha256=815162ac09f5e6e914f6a5cc5ad6c8a08f

In [ ]:
import torch
from sentence_transformers import SentenceTransformer

# set device to GPU if available
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# load the table embedding model from huggingface models hub
retriever = SentenceTransformer("deepset/all-mpnet-base-v2-table", device=device)
retriever

.gitattributes:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.42k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/576 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/345 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

SentenceTransformer(
  (0): Transformer({'max_seq_length': 384, 'do_lower_case': False}) with Transformer model: MPNetModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
)

In [ ]:
from transformers import pipeline, TapasTokenizer, TapasForQuestionAnswering

model_name = "google/tapas-base-finetuned-wtq"
# load the tokenizer and the model from huggingface model hub
tokenizer = TapasTokenizer.from_pretrained(model_name, drop_rows_to_fit=True)
model = TapasForQuestionAnswering.from_pretrained(model_name, local_files_only=False)
# load the model and tokenizer into a question-answering pipeline
pipe = pipeline("table-question-answering",  model=model, tokenizer=tokenizer, device=device)

In [ ]:
import pinecone

from google.colab import userdata
key = userdata.get('pinecone')

# connect to pinecone environment
pinecone.init(
    api_key=key,
    environment="gcp-starter"  # find next to API key in console
)

In [ ]:
# you can choose any name for the index
index_name = "personal-roboadvisor"

# check if the table-qa index exists
if index_name not in pinecone.list_indexes():
    # create the index if it does not exist
    pinecone.create_index(
        index_name,
        dimension=768,
        metric="cosine"
    )

# connect to table-qa index we created
index = pinecone.Index(index_name)

In [ ]:
def _preprocess_tables(tables: list):
    processed = []
    # loop through all tables
    for table in tables:
        # convert the table to csv and
        processed_table = "\n".join([table.to_csv(index=False)])
        # add the processed table to processed list
        processed.append(processed_table)
    return processed

## Setup Personal Finance

In [ ]:
import pandas as pd
pd.set_option('float_format', '{:f}'.format)
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Load Data and Drop Null Columns
base_url = ''
workbook_id = ''
df = pd.read_csv(f'{base_url}{workbook_id}/export?format=csv')

# Change Date to datetime for easier slicing
df['Date'] = pd.to_datetime(df['Date'], format='%m/%d/%Y')
df['day_of_week'] = df['Date'].dt.day_name()
df = df.loc[(df['Date'] >= '09/01/2023')]

df['Date'] = df['Date'].astype(str)

# Change Amount to Float data type
df['Amount'] = [str(i).replace(",", "") for i in df['Amount']]

df = df.replace(np.nan, '')

In [ ]:
df.dtypes

Date                object
Description         object
Category            object
Category Type       object
Amount              object
Account             object
Account #           object
Institution         object
Month               object
Week                object
Full Description    object
Date Added          object
day_of_week         object
dtype: object

In [ ]:
df.isna().sum()

Date                0
Description         0
Category            0
Category Type       0
Amount              0
Account             0
Account #           0
Institution         0
Month               0
Week                0
Full Description    0
Date Added          0
day_of_week         0
dtype: int64

In [ ]:
df.head()

## Combine Personal Finance & Q&A

In [ ]:
tables = [df]

# format all the dataframes in the tables list
processed_tables = _preprocess_tables(tables)

In [ ]:
from tqdm.auto import tqdm

# we will use batches of 64
batch_size = 64

for i in tqdm(range(0, len(processed_tables), batch_size)):
    # find end of batch
    i_end = min(i+batch_size, len(processed_tables))
    # extract batch
    batch = processed_tables[i:i_end]
    # generate embeddings for batch
    emb = retriever.encode(batch).tolist()
    # create unique IDs ranging from zero to the total number of tables in the dataset
    ids = [f"{idx}" for idx in range(i, i_end)]
    # add all to upsert list
    to_upsert = list(zip(ids, emb))
    # upsert/insert these records to pinecone
    _ = index.upsert(vectors=to_upsert)

# check that we have all vectors in index
index.describe_index_stats()

  0%|          | 0/1 [00:00<?, ?it/s]

{'dimension': 768,
 'index_fullness': 1e-05,
 'namespaces': {'': {'vector_count': 1}},
 'total_vector_count': 1}

In [ ]:
query = "What category type is a fee?"

# generate embedding for the query
xq = retriever.encode([query]).tolist()
# query pinecone index to find the table containing answer to the query
result = index.query(xq, top_k=1)

result

{'matches': [{'id': '0', 'score': 0.502352178, 'values': []}], 'namespace': ''}

In [ ]:
id = int(result["matches"][0]["id"])
tables[id].head()

In [ ]:
pipe(table=tables[id], query=query, truncation=True)

{'answer': 'Expense',
 'coordinates': [(2, 3)],
 'cells': ['Expense'],
 'aggregator': 'NONE'}

# Querying

In [ ]:
def query_pinecone(query):
    # generate embedding for the query
    xq = retriever.encode([query]).tolist()
    # query pinecone index to find the table containing answer to the query
    result = index.query(xq, top_k=1)
    # return the relevant table from the tables list
    return tables[int(result["matches"][0]["id"])]

In [ ]:
def get_answer_from_table(table, query):
    # run the table and query through the question-answering pipeline
    answers = pipe(table=table, query=query)
    return answers

In [ ]:
query = "What category type is a fee?"
table = query_pinecone(query)
answer = get_answer_from_table(table, query)

{'answer': 'Expense',
 'coordinates': [(2, 3)],
 'cells': ['Expense'],
 'aggregator': 'NONE'}

In [ ]:
print('Query:',query,'Answer:',answer['answer'])

Query:  What category type is a fee? Answer:  Expense
